In [20]:
import pandas as pd
import random

https://gengo.ai/datasets/15-free-sentiment-analysis-datasets-for-machine-learning/

In [21]:
import json
i = 0
reviews_lst_good= []
reviews_lst_bad = []


with open('C:\\Users\\denis.pekhterev\\Downloads\\Experiment_old\\Books_5.json', 'r') as f:
        for line in f:
            if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                reviews_lst_good.append (json.loads(line))
            
            elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                reviews_lst_bad.append (json.loads(line))

            if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                break
                
random.shuffle(reviews_lst_good)
random.shuffle(reviews_lst_bad)

In [22]:
reviewerID_lst = []
reviewerName_lst = []
reviewText_lst = []
overall_lst = []
summary_lst = []

# reviews_lst [0]['overall']<5 #['asin']for 

for i in reviews_lst_good:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
for i in reviews_lst_bad:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
df = pd.DataFrame ()

df['reviewerID'] = reviewerID_lst
# df['reviewerName'] = reviewerName_lst
df['reviewText'] = reviewText_lst
df['overall'] = overall_lst
df['summary'] = summary_lst

In [23]:
df ['mark'] = -1
df.loc[df['overall'] > 3,'mark'] = 1

print (len(df[df['mark'] > 0]))

500001


In [24]:
df[(df['mark'] != -1)&(df['mark'] != 1)]

,reviewerID,reviewText,overall,summary,mark


In [25]:
reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])

In [26]:
# list(df[df['overall'] > 3]['reviewText'][:2])
reviews_train [:2]

["Archer tells us about Whitechapel at the time of World War I. He misses out that Whitechapel was predominantly Jewish and it was here that the Jewish Legion (part of the British Army) was organized, paraded (right down Charlie's street) and became the model for the Jewish Brigade of World War II.",
 "Even though I agree with a few people the SEP books have can be a little formalistic I rated this books so high due to the great humor. I haven't laughed so hard with any other book. The heroine is written with such wit and unabashed sarcasm that you can't help laugh. The plot is fun and steamy, and yes if you have read many of SEP books a little familiar. But it's worth the read if only for the great dialog.I listened to this on Audible.com and found the narrator only helped add to the fun of the book."]

In [27]:
import re
from nltk.corpus import stopwords 

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [29]:

y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

# for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
#     lr = LogisticRegression(C=c)
#     lr.fit(X_train, y_train)
#     print ("Accuracy for C=%s: %s" 
#            % (c, accuracy_score(y_val, lr.predict(X_val))))

In [30]:
final_model = LogisticRegression(C=0.5)
final_model.fit(X,  y)
print ("Final Accuracy: %s" 
       % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                        , final_model.predict(X_test)))

Final Accuracy: 0.91145


In [31]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

('klausner', 4.147142370815207)
('ariely', 2.813309136917886)
('everneath', 2.725227604823884)
('spong', 2.6095583721733715)
('maxon', 2.527694638939319)
('booksneeze', -3.473470371960414)
('waterbrook', -2.7344756508000025)
('scarpetta', -2.6240425559570726)
('killgore', -2.62346383374639)
('booksneezecom', -2.483382630085467)


In [13]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:120]:
    print (best_positive)

writers = ['klausner','rollins','ehrman','gaiman','spong','olaf','steinbeck','harpercollins','discworld',
'kingsolver','pratchett','maxon']

('klausner', 4.200585683240843)
('ariely', 2.8356831815044803)
('everneath', 2.6124595643870463)
('maxon', 2.548042693956325)
('spong', 2.503794565750101)
('olaf', 2.335943150736356)
('ehrman', 2.2755094891117817)
('pleasantly', 2.2397345696720072)
('aria', 2.227326464942671)
('margolin', 2.209052535420335)
('erdrich', 2.016718935236432)
('wynne', 2.001600642188314)
('proses', 1.9956662863936512)
('tamani', 1.9886643508909623)
('kaiden', 1.9801171303234608)
('hobb', 1.9683015005540692)
('maclean', 1.952409460033418)
('zinsser', 1.94943085254741)
('lobdell', 1.9294905306995673)
('argeneau', 1.925562957231128)
('halpern', 1.915425482499514)
('wallis', 1.9136862005436155)
('rollins', 1.9082115838869715)
('humbert', 1.9015820908296284)
('refreshing', 1.9004044314727193)
('gaiman', 1.8980815479534694)
('quibble', 1.8979643410512592)
('harpercollins', 1.8927034055101044)
('alafair', 1.8753738127073951)
('dany', 1.853341642609548)
('gitomer', 1.8495983741606035)
('septimus', 1.842669829383629

# Analyse reviews

In [15]:
import json
import random
import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")


def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews

def classify_it(filename, parametr): 
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []


    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                    break
    
    random.shuffle(reviews_lst_good)
    random.shuffle(reviews_lst_bad)
                    
    reviewerID_lst = []
    reviewerName_lst = []
    reviewText_lst = []
    overall_lst = []
    summary_lst = []

    # reviews_lst [0]['overall']<5 #['asin']for 

    for i in reviews_lst_good:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    for i in reviews_lst_bad:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    df = pd.DataFrame ()

    df['reviewerID'] = reviewerID_lst
    # df['reviewerName'] = reviewerName_lst
    df['reviewText'] = reviewText_lst
    df['overall'] = overall_lst
    df['summary'] = summary_lst
    
    df ['mark'] = 0
    df.loc[df['overall'] > 3,'mark'] = 1
    
    
    
    reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
    reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])



    reviews_train_clean = preprocess_reviews(reviews_train)
    reviews_test_clean = preprocess_reviews(reviews_test)
    
    cv = CountVectorizer(binary=True)
    cv.fit(reviews_train_clean)
    X = cv.transform(reviews_train_clean)
    X_test = cv.transform(reviews_test_clean)
    
    y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, train_size = 0.75
    )
    
    print ('Подбор параметра для: ', filename)
    
    for c in [0.01, 0.05, 0.25, 0.5, 1]:
        lr = LogisticRegression(C=c)
        lr.fit(X_train, y_train)
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, lr.predict(X_val))))
    
    
    final_model = LogisticRegression(C=parametr)
    final_model.fit(X,  y)
    print ("Final Accuracy: %s" 
           % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                            , final_model.predict(X_test)))
    
    feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
        )
    }
    for best_positive in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1], 
        reverse=True)[:5]:
        print (best_positive)
    for best_negative in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1])[:5]:
        print (best_negative)
        
    del reviews_lst_good
    del reviews_lst_bad

In [16]:
import os
import gc

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints':
        print (i)
        print ('')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.5)
        gc.collect()
        

Health_and_Personal_Care_5.json

Подбор параметра для:  G:\Мой диск\Work\Experiment\Health_and_Personal_Care_5.json
Accuracy for C=0.01: 0.9375511763742562
Accuracy for C=0.05: 0.94242316720345
Accuracy for C=0.25: 0.9450161034990993
Accuracy for C=0.5: 0.9445930454719144
Accuracy for C=1: 0.9438561056826246
Final Accuracy: 0.73015
('deducted', 1.7656824591106735)
('pleasantly', 1.656820137862464)
('pleased', 1.6369400592065257)
('skeptical', 1.5956846843153587)
('highly', 1.5722485210054)
('disappointing', -2.5610881508681285)
('worthless', -2.4379621815169608)
('disappointment', -2.143615023529117)
('unacceptable', -2.0601684486255207)
('poorly', -1.905376850288279)
Movies_and_TV_5.json

Подбор параметра для:  G:\Мой диск\Work\Experiment\Movies_and_TV_5.json
Accuracy for C=0.01: 0.907810879772571
Accuracy for C=0.05: 0.9136597187430822
Accuracy for C=0.25: 0.9151918349275886
Accuracy for C=0.5: 0.914830651644549
Accuracy for C=1: 0.9139975998788288
Final Accuracy: 0.88745
('coulardea

In [17]:
import os
import gc

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints':
        print (i)
        print ('')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.25)
        gc.collect()
        

Health_and_Personal_Care_5.json

Подбор параметра для:  G:\Мой диск\Work\Experiment\Health_and_Personal_Care_5.json
Accuracy for C=0.01: 0.9362683552595665
Accuracy for C=0.05: 0.9409765816911404
Accuracy for C=0.25: 0.9428871663300399
Accuracy for C=0.5: 0.94253234346853
Accuracy for C=1: 0.9417408155467002
Final Accuracy: 0.72545
('pleased', 1.5682731570972217)
('hesitate', 1.5175314175031036)
('glad', 1.450800831389244)
('excellent', 1.4128644601444649)
('complaints', 1.4126178401834528)
('disappointing', -2.2840623587693756)
('worthless', -2.0835303820172033)
('disappointment', -1.9798090638320425)
('useless', -1.8005429485885187)
('unacceptable', -1.6859019789724752)
Movies_and_TV_5.json

Подбор параметра для:  G:\Мой диск\Work\Experiment\Movies_and_TV_5.json
Accuracy for C=0.01: 0.907810879772571
Accuracy for C=0.05: 0.9135315569329714
Accuracy for C=0.25: 0.9153025201272297
Accuracy for C=0.5: 0.9148539537918419
Accuracy for C=1: 0.9137529273322537
Final Accuracy: 0.88665
('coul

In [18]:
import json
import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")


def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews

def classify_it(filename, parametr): 
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []


    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                    break
    reviews_lst_good = random.shuffle(reviews_lst_good)
    reviews_lst_bad = random.shuffle(reviews_lst_bad)  
    
    reviewerID_lst = []
    reviewerName_lst = []
    reviewText_lst = []
    overall_lst = []
    summary_lst = []

    # reviews_lst [0]['overall']<5 #['asin']for 

    for i in reviews_lst_good:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    for i in reviews_lst_bad:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    df = pd.DataFrame ()

    df['reviewerID'] = reviewerID_lst
    # df['reviewerName'] = reviewerName_lst
    df['reviewText'] = reviewText_lst
    df['overall'] = overall_lst
    df['summary'] = summary_lst
    
    df ['mark'] = 0
    df.loc[df['overall'] > 3,'mark'] = 1
    
    reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
    reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])



    reviews_train_clean = preprocess_reviews(reviews_train)
    reviews_test_clean = preprocess_reviews(reviews_test)
    
    cv = TfidfVectorizer(binary=True)
    cv.fit(reviews_train_clean)
    X = cv.transform(reviews_train_clean)
    X_test = cv.transform(reviews_test_clean)
    
    y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, train_size = 0.75
    )
    
    print ('Подбор параметра для: ', filename)
    print ('    ')
    
    for c in [0.01, 0.05, 0.25, 0.5, 1]:
        lr = LogisticRegression(C=c)
        lr.fit(X_train, y_train)
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, lr.predict(X_val))))
    
    
    final_model = LogisticRegression(C=parametr)
    final_model.fit(X,  y)
    print ("Final Accuracy: %s" 
           % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                            , final_model.predict(X_test)))
    
    feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
        )
    }
    for best_positive in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1], 
        reverse=True)[:5]:
        print (best_positive)
    for best_negative in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1])[:5]:
        print (best_negative)
        
    del reviews_lst_good
    del reviews_lst_bad

In [19]:
import os
import gc

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints':
        print (i)
        print ('   ')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.25)
        gc.collect()
        print ('   ')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.5)
        

Health_and_Personal_Care_5.json
   


TypeError: 'NoneType' object is not iterable

In [ ]:
import random

list = [20, 16, 10, 5];
random.shuffle(list)
print ("Reshuffled list : ",  list)

In [ ]:
# BackUP

In [ ]:
# df = pd.read_json('Books_5.json')
# import ijson
# filename = "Books_5.json"
# with open(filename, 'r') as f:
#     objects = ijson.items(f, 'meta.view.columns.item')
#     columns = list(objects)
#     print (objects)
# handle = open("Books_5.json", "r")

# while True:
#     data = handle.read(20000)
#     if not data:
#         break